In [ ]:
import xarray as xr
import matplotlib.pyplot as plt

In [ ]:
def read_ig_nc_file(stock_symbol):
  nc_path = f'Integrated_Gradients/{stock_symbol}/save_rolltrain_Integrate_Gradient_'+stock_symbol
  Integrate_Gradient_X = xr.open_dataset(f"{nc_path}_X.nc").to_array().resample({'Datetime':'M'}).mean()
  Integrate_Gradient_last_w = xr.open_dataset(f"{nc_path}_last_w.nc").to_array().resample({'Datetime':'M'}).mean()
  return Integrate_Gradient_X,Integrate_Gradient_last_w

In [ ]:
import plotly.express as px

feature_labels = {
    'close':'Close',
    'high':'High',
    'low':'Low', 
    'open':'Open',
    'volume':'Volume', 
    'beta_2':'Beta 2-year',
    'alpha_2':'Alpha 2-year', 
    'beta_5':'Beta 5-year', 
    'alpha_5':'Alpha 5-year', 
    'MA_50':'Moving Average 50-day', 
    'MA_200':'Moving Average 200-day', 
    'EMA_50':'Exponential Moving Average 50-day', 
    'EMA_200':'Exponential Moving Average 200-day',
    'RSI_14':'Relative Strength Indicator 14-day', 
    'ATR_14':'Average True Range 14-day',
    'DGS1MO': '1-Month Treasury',
    'DGS3MO': '3-Month Treasury',
    'DGS1': '1-Year Treasury',
    'DGS3': '3-Year Treasury',
    'DGS10': '10-Year Treasury',
    'T10Y2Y': '10-2 Treasury Yield Spread',
    'CPALTT01USM657N': 'Consumer Price Index',
    'DLTIIT': 'Long-Term Average Yield',
    'T5YIFR': 'Forward Inflation Expectation Rate',
    'T10YIE': '10-Year Breakeven Inflation Rate',
    'MICH': 'MICH Inflation Expectation',
    'WM1NS': 'M1 Money Stock',
    'WM2NS': 'M2 Money Stock',
    'BOGMBASE': 'Monetary Base',
    'UNRATE': 'US Unemployment Rate',
    'JTSJOL': 'Job Openings: Total Nonfarm',
    'IC4WSA': 'Moving Average of Initial Claims',
    'INDPRO': 'Industrial Production',
    'MRTSSM44X72USS': 'Retail Sales',
    'UMCSENT': 'MICH Consumer Sentiment',
    'VIXCLS': 'CBOE Volatility Index',
    'STLFSI2': 'Financial Stress Index',
    'DCOILBRENTEU': 'Crude Oil Prices',
    'GOLDAMGBD228NLBM': 'Gold Fixing Price',
    'GEPUCURRENT': 'Economic Policy Uncertainty Index',
}

def plot_agg_ig(Integrate_Gradient_X,stock_symbol):
  agg_ig = Integrate_Gradient_X.mean(dim='Window').mean(dim='Stock Symbols')

  features_to_drop = []
  for feature_idx in range(agg_ig.shape[2]):
    check_small_values=abs(agg_ig[0,:,feature_idx])>1e-4
    if any(check_small_values)==False:
      features_to_drop.append(str(agg_ig.coords['Features'].values[feature_idx]))
  agg_ig = agg_ig.drop_sel(Features=features_to_drop)

  agg_ig_df = agg_ig.squeeze().to_dataframe(name='agg_ig').reset_index()[['Datetime','Features','agg_ig']].replace({"Features": feature_labels})

  fig = px.line(agg_ig_df,x='Datetime', y='agg_ig', color='Features',labels={'Datetime':'','agg_ig':'Integrated Gradients'},title=f"Integrated Gradients by Features for {stock_symbol}")

  fig.update_layout(
      yaxis = dict(
          showexponent = 'all',
          exponentformat = 'e'
      )
  )

  fig.show()

  feature_df = Integrate_Gradient_X.mean('Stock Symbols').squeeze().to_dataframe(name='ig_values').reset_index()[['Datetime','Window','Features','ig_values']]\
                                    .replace({"Features": feature_labels})

  fig = px.box(feature_df,y='ig_values',x='Features',animation_frame='Window',height=750)
  fig.update_xaxes(tickangle=90)
  fig['layout']['sliders'][0]['pad']=dict(r= 10, t= 200,)
  fig['layout']['updatemenus'][0]['pad']=dict(r= 10, t= 200)
  fig.show()

  stock_symbol_df = Integrate_Gradient_X.mean('Features').squeeze().to_dataframe(name='ig_values').reset_index()[['Datetime','Window','Stock Symbols','ig_values']]
  fig = px.box(stock_symbol_df,y='ig_values',x='Stock Symbols',animation_frame='Window',height=750)

  fig.show()

In [ ]:
stock_symbol = 'XLK'
Integrate_Gradient_X,Integrate_Gradient_last_w = read_ig_nc_file(stock_symbol)
plot_agg_ig(Integrate_Gradient_X,stock_symbol)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
stock_symbol = 'XLB'
Integrate_Gradient_X,Integrate_Gradient_last_w = read_ig_nc_file(stock_symbol)
plot_agg_ig(Integrate_Gradient_X,stock_symbol)

Output hidden; open in https://colab.research.google.com to view.